<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [14]</a>'.</span>

# DB2-Salesforce connector: Citation mapping to users

In [1]:
# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'Name'
object_id = 'contact_citation_asso__c'

# login credentials
sql_login_params = {"username": "wang2506_ro", "password": "fnVnwcCS7iT45EsA"}
sf_login_params = {
    "grant_type": "password",
    "client_id": "3MVG95jctIhbyCppj0SNJ75IsZ1y8UPGZtSNF4j8FNVXz.De8Lu4jHm3rjRosAtsHy6qjHx3i4S_QbQzvBePG",
    "client_secret": "D1623C6D3607D4FC8004B92C761DFB6C1F70CCD129C5501E357028DFA00F5764",
    "username":"wang2506@purdue.edu",
    "password":"purdue99272844vp8Z6qq0teuPQmirGaaPAt4",     
} 
    #"username": "wang159-4j1v@force.com",
    #"password": "napoleon0eZ3PQpQqE3C3z4wWqEqKGhQ8",

In [2]:
import sys
sys.path.append('/home/users/wang2506/nanohub_salesforce_integ/salesforce')
    
import pandas as pd
import time
import datetime

from DB2SalesforceAPI import DB2SalesforceAPI

## Obtain tool information from DB2 

In [3]:
# connect with DB2
import sqlalchemy as sql

engine = sql.create_engine('mysql+pymysql://%s:%s@127.0.0.1/nanohub' \
                                               %(sql_login_params['username'], sql_login_params['password']))

# Citations
sql_query = "select * from jos_citations"

citations_df = pd.read_sql_query(sql_query, engine)

# Citation authors
sql_query = "select * from jos_citations_authors;"

authors_df = pd.read_sql_query(sql_query, engine)

In [4]:
# replace N/A and empty values with None
authors_df.organization.replace('N/A', '', inplace=True)

In [5]:
authors_df.columns

Index(['id', 'cid', 'author', 'authorid', 'uidNumber', 'ordering', 'givenName',
       'middleName', 'surname', 'organization', 'org_dept', 'orgtype',
       'countryresident', 'email', 'ip', 'host', 'countrySHORT', 'countryLONG',
       'ipREGION', 'ipCITY', 'ipLATITUDE', 'ipLONGITUDE', 'in_network',
       'orcid', 'research_id', 'gscholar_id', 'scopus_id', 'researchgate_id',
       'notes'],
      dtype='object')

In [6]:
# combining authors and citations
ac_df = pd.merge(citations_df[['id']], authors_df[['cid', 'uidNumber','id','organization','orcid','research_id','gscholar_id']], how='inner',\
                 left_on='id', right_on='cid', suffixes=['_c', '_a'])\
                .drop(columns='id_c')

ac_df.rename(columns={'cid':'citation_ID', 'uidNumber':'nanohub_user_id', 'id_a':'author_id'}, inplace=True)

# display
ac_df.head(3)

,citation_ID,nanohub_user_id,author_id,organization,orcid,research_id,gscholar_id
0,10000003,10676,4613,Purdue University,None,None,None
1,10000003,0,4614,Purdue University,None,None,None
2,10000003,37033,4615,Purdue University,None,None,None


In [7]:
# Obtain company domain information from DB2
domain_df = pd.read_sql_query("select name as domain_name, domain, industry, `size range` as size, country \
from wang159_myrmekes.companies_email_domain", engine)

# make sure domain is unique and drop NaN
domain_df = domain_df.drop_duplicates(subset='domain', keep='first')

In [8]:
import nltk
#nltk.download('stopwords')

from nltk.corpus import stopwords
s_words = stopwords.words('english')

In [9]:
import re

def clean_domain(this_domain_name):
    
    if not this_domain_name:
        return None
    
    # Replace all non-alphanumeric characters with space
    this_domain_name = re.sub("[^0-9a-zA-Z]+", " ", this_domain_name.lower())
    name_list = this_domain_name.split(' ')
        
    # remove all stop words
    name_list = ['' if x in s_words else x for x in name_list]
    
    return set(filter(None, name_list))

In [10]:
# clean domain name
domain_df['domain_cleaned_set'] = domain_df.domain_name.apply(clean_domain)

# hash cleaned domain name
domain_df['domain_cleaned_hash'] = domain_df['domain_cleaned_set'].apply(lambda x: '-'.join(sorted(list(x))) if x else None)

In [11]:
# clean domain name
ac_df['domain_cleaned_set'] = ac_df.organization.apply(clean_domain)

# hash cleaned domain name
ac_df['domain_cleaned_hash'] = ac_df['domain_cleaned_set'].apply(lambda x: '-'.join(sorted(list(x))) if x else None)

In [12]:
# get domain subset that contains domain_cleaned_hash in authors_df
domain_subset_df = domain_df[domain_df.domain_cleaned_hash.isin(ac_df.domain_cleaned_hash.unique())]\
                                        [['domain', 'domain_cleaned_hash']]
domain_subset_df = domain_subset_df[domain_subset_df.domain.notnull()&domain_subset_df.domain_cleaned_hash.notnull()]

domain_subset_all_df = domain_subset_df.rename(columns={'domain_cleaned_hash':'domain_hash'}).drop_duplicates()

In [13]:
# attempt direct join by hash
derived_authors_df = pd.merge(ac_df, domain_subset_all_df\
                              ,how='left', left_on='domain_cleaned_hash', right_on='domain_hash')\
                              .drop('domain_hash', axis=1)\
                              .rename(columns={'domain':'domain_by_citation'})

# display
derived_authors_df.sample(5).T

,7823,2892,6043,4646,16264
citation_ID,2967,10000671,10001464,10001156,3186
nanohub_user_id,24835,0,3482,0,0
author_id,14183757,7468,10593,9196,14176233
organization,Purdue University,Purdue University,Purdue University,Purdue University,Synopsys Inc
orcid,,None,None,None,
research_id,,None,None,None,
gscholar_id,,None,None,None,
domain_cleaned_set,"{university, purdue}","{university, purdue}","{university, purdue}","{university, purdue}","{inc, synopsys}"
domain_cleaned_hash,purdue-university,purdue-university,purdue-university,purdue-university,inc-synopsys
domain_by_citation,purdue.edu,purdue.edu,purdue.edu,purdue.edu,NaN


<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [14]:
# rank the nanoHUB profile institutions that cannot be directly matched by occurance

derived_authors_subset_df = derived_authors_df[(derived_authors_df.domain_cleaned_hash.notnull() \
                                                                & derived_authors_df.domain_by_citation.isna())]

most_common_sets = derived_authors_subset_df.domain_cleaned_set.value_counts()

# display
most_common_sets.sort_values(ascending=False)

TypeError: unhashable type: 'set'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 1652, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'set'


{university, south, wales, new}              263
{institute, technology, california}          229
{technology, university, delft}              153
{systems, raytheon}                          103
{state, pennsylvania, university}             96
                                            ... 
{china, branch, crosslight, sofware, inc}      1
{incheon, university, national}                1
{consulting, omega, enterprises}               1
{veritas, north, bureau, america}              1
{pisa, di, uiversita}                          1
Name: domain_cleaned_set, Length: 962, dtype: int64

In [15]:
# attempt to reduce domain_df size to speedup Jaccard calculation
from itertools import chain

all_word_set = set(chain.from_iterable(derived_authors_subset_df.domain_cleaned_set.values))

# select the domain_df entry with words from all_word_set
domain_subset_df = domain_df[domain_df.domain_cleaned_set.notnull()]
domain_subset_df = domain_subset_df[domain_subset_df.domain_cleaned_set\
                             .apply(lambda x: True if len(x.intersection(all_word_set)) > 0 else False)]

domain_subset_df = domain_subset_df[['domain_cleaned_set', 'domain']]

In [16]:
# Time consuming part: calculating Jaccard similarity score
def get_jaccard_score(a,b):
    
    if (not a) & (not b):
        return 0
    
    score = len(a.intersection(b))/len(a.union(b))
    
    return score


for this_set in most_common_sets.index:
    # calculate the Jaccard similarity
    max_index = domain_subset_df.domain_cleaned_set.apply(lambda x: get_jaccard_score(x, this_set)).idxmax()

    derived_authors_df.loc[derived_authors_df.domain_cleaned_set == this_set, 'domain_by_citation'] \
                = domain_subset_df.loc[max_index, 'domain']

In [17]:
# display
derived_authors_df.sample(5).T

,354,5116,2367,428,143
citation_ID,10000077,10001257,10000556,10000096,10000035
nanohub_user_id,2862,3482,14521,0,20857
author_id,4965,9666,6947,5037,4754
organization,Purdue University,Purdue University,California Institute of Technology,University of California,Delft University of Technology
orcid,None,None,None,None,None
research_id,None,None,None,None,None
gscholar_id,None,None,None,None,None
domain_cleaned_set,"{university, purdue}","{university, purdue}","{institute, technology, california}","{university, california}","{technology, university, delft}"
domain_cleaned_hash,purdue-university,purdue-university,california-institute-technology,california-university,delft-technology-university
domain_by_citation,purdue.edu,purdue.edu,scitech.edu,universityofcalifornia.edu,sifedelft.nl


## Obtain Salesforce IDs

In [18]:
# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

Obtained Salesforce access token ...... True


In [19]:
# query the Salesforce IDs for contacts and citations. when updating junction objects, these IDs must be used

# get Salesforce ID for contacts
sf_userID_df = db_s.query_data('SELECT Id, nanoHUB_user_ID__c FROM Contact where nanoHUB_user_ID__c != NULL')

sf_userID_df['nanoHUB_user_ID__c'] = sf_userID_df['nanoHUB_user_ID__c'].astype('int')

[Success] Bulk job creation successful. Job ID = 7505w00000TmJghAAF


{"id":"7505w00000TmJghAAF","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2021-01-21T07:34:31.000+0000","systemModstamp":"2021-01-21T07:34:31.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}


{"id":"7505w00000TmJghAAF","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2021-01-21T07:34:31.000+0000","systemModstamp":"2021-01-21T07:34:32.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":0}


{"id":"7505w00000TmJghAAF","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2021-01-21T07:34:31.000+0000","systemModstamp":"2021-01-21T07:34:44.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":236376,"retries":0,"totalProcessingTime":11358}
[Success] Bulk job completed successfully.


In [20]:
# get Salesforce ID for citations
sf_citationID_df = db_s.query_data('SELECT Id, Record_ID__c FROM nanoHUB_citations__c')

sf_citationID_df['Record_ID__c'] = sf_citationID_df['Record_ID__c'].astype('int')

[Success] Bulk job creation successful. Job ID = 7505w00000TmJgmAAF


{"id":"7505w00000TmJgmAAF","operation":"query","object":"nanoHUB_citations__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-01-21T07:34:57.000+0000","systemModstamp":"2021-01-21T07:34:57.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":0}


{"id":"7505w00000TmJgmAAF","operation":"query","object":"nanoHUB_citations__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-01-21T07:34:57.000+0000","systemModstamp":"2021-01-21T07:34:58.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":2507,"retries":0,"totalProcessingTime":330}
[Success] Bulk job completed successfully.


In [21]:
# get Salesforce ID for organizations
sf_orgID_df = db_s.query_data('SELECT Id, Domain__c FROM organization__c')

[Success] Bulk job creation successful. Job ID = 7505w00000TmJhBAAV


{"id":"7505w00000TmJhBAAV","operation":"query","object":"organization__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-01-21T07:35:09.000+0000","systemModstamp":"2021-01-21T07:35:09.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}


{"id":"7505w00000TmJhBAAV","operation":"query","object":"organization__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-01-21T07:35:09.000+0000","systemModstamp":"2021-01-21T07:35:10.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":6365,"retries":0,"totalProcessingTime":421}
[Success] Bulk job completed successfully.


## Match data with Salesforce format

In [22]:
# valid citation - nanohub user links
ac_tolink_df = derived_authors_df[derived_authors_df.nanohub_user_id != 0]

display(ac_tolink_df.head(2))
display(sf_citationID_df.head(2))
display(sf_userID_df.head(2))
display(sf_orgID_df.head(2))

,citation_ID,nanohub_user_id,author_id,organization,orcid,research_id,gscholar_id,domain_cleaned_set,domain_cleaned_hash,domain_by_citation
0,10000003,10676,4613,Purdue University,None,None,None,"{university, purdue}",purdue-university,purdue.edu
2,10000003,37033,4615,Purdue University,None,None,None,"{university, purdue}",purdue-university,purdue.edu


,Id,Record_ID__c
0,a0t5w000008p7pSAAQ,1279
1,a0t5w000008p7pTAAQ,1535


,Id,nanoHUB_user_ID__c
0,0035w000031Vsp1AAC,998
1,0035w000031Vsp2AAC,1683


,Domain__c,Id
0,upenn.edu,a0r5w00000V42c0AAB
1,fer.hr,a0r5w00000V42c1AAB


In [23]:
# merge SF citation and contact IDs into user-citation DF
ac_tolink_df = pd.merge(ac_tolink_df, sf_citationID_df, how='inner', left_on='citation_ID', right_on='Record_ID__c')\
                           .rename(columns={'Id':'SF_ID_citation'})
                                            
ac_tolink_df = pd.merge(ac_tolink_df, sf_userID_df, how='inner', left_on='nanohub_user_id', right_on='nanoHUB_user_ID__c')\
                           .rename(columns={'Id':'SF_ID_contact'})

ac_tolink_df = pd.merge(ac_tolink_df, sf_orgID_df, how='left', left_on='domain_by_citation', right_on='Domain__c')\
                           .rename(columns={'Id':'SF_ID_organization'})

# display
ac_tolink_df.head(2)

,citation_ID,nanohub_user_id,author_id,organization,orcid,research_id,gscholar_id,domain_cleaned_set,domain_cleaned_hash,domain_by_citation,SF_ID_citation,Record_ID__c,SF_ID_contact,nanoHUB_user_ID__c,Domain__c,SF_ID_organization
0,2035,65635,14184931,Purdue University,,,,"{university, purdue}",purdue-university,purdue.edu,a0t5w000008p7qAAAQ,2035,0035w000031Vu0yAAC,65635,purdue.edu,a0r5w00000V42cCAAR
1,2856,65635,14184790,Purdue University,,,,"{university, purdue}",purdue-university,purdue.edu,a0t5w000008p8OtAAI,2856,0035w000031Vu0yAAC,65635,purdue.edu,a0r5w00000V42cCAAR


In [24]:
# create a new column for object name
ac_tolink_df['Name'] = ac_tolink_df.apply(lambda x: '%d_%d'%(x.nanoHUB_user_ID__c, x.Record_ID__c), axis=1)

In [25]:
df_sf = pd.DataFrame()

# Make sure NaN and NaT values are taken care of here
df_sf['Name']         = ac_tolink_df['Name']
df_sf['Contact__c'] = ac_tolink_df['SF_ID_contact']
df_sf['Citation__c'] = ac_tolink_df['SF_ID_citation']

df_sf['gscholar_id__c'] = ac_tolink_df['gscholar_id'].fillna('')
df_sf['ORCID__c'] = ac_tolink_df['orcid'].fillna('')
df_sf['Organization__c'] = ac_tolink_df['SF_ID_organization']
df_sf['Organization_nanohub__c'] = ac_tolink_df['organization'].fillna('')
df_sf['research_id__c'] = ac_tolink_df['research_id'].fillna('')

sf_original_fields = df_sf.columns

# display
df_sf.head(2).T

,0,1
Name,65635_2035,65635_2856
Contact__c,0035w000031Vu0yAAC,0035w000031Vu0yAAC
Citation__c,a0t5w000008p7qAAAQ,a0t5w000008p8OtAAI
gscholar_id__c,,
ORCID__c,,
Organization__c,a0r5w00000V42cCAAR,a0r5w00000V42cCAAR
Organization_nanohub__c,Purdue University,Purdue University
research_id__c,,


## To Salesforce Sales Cloud CRM

In [26]:
# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

# specify Salesforce object ID and external ID
db_s.object_id = 'contact_citation_asso__c'
db_s.external_id = 'Name'

Obtained Salesforce access token ...... True


In [27]:
# send data to Salesforce
db_s.send_data(df_sf)

[Success] Bulk job creation successful. Job ID = 7505w00000TmJbiAAF
hello


[Success] CSV upload successful. Job ID = 7505w00000TmJbiAAF


[Success] Closing job successful. Job ID = 7505w00000TmJbiAAF


In [28]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_status())

{'apexProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apiVersion': 47.0,
 'columnDelimiter': 'COMMA',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-01-21T07:35:23.000+0000',
 'externalIdFieldName': 'Name',
 'id': '7505w00000TmJbiAAF',
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'numberRecordsFailed': 0,
 'numberRecordsProcessed': 0,
 'object': 'contact_citation_asso__c',
 'operation': 'upsert',
 'retries': 0,
 'state': 'InProgress',
 'systemModstamp': '2021-01-21T07:35:24.000+0000',
 'totalProcessingTime': 0}


In [29]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_failed_results())

''
